In [2]:
import sys
from custom_gromacstopfile import CustomGromacsTopFile
import backward

from simtk.openmm import app
import simtk.openmm as mm
import simtk.unit as u
import openmmtools.integrators as toolIntegrators
from openmmtools.integrators import GradientDescentMinimizationIntegrator
from openmmtools.integrators import ThermostatedIntegrator

import numpy as np
%config Completer.use_jedi=False

In [3]:
%matplotlib inline
# widget for jupyterlab, notebook for jupyter-notebooks, inline for debugging
%load_ext autoreload
%autoreload 2

## Files

In [23]:
# input
cg_structure = "/home/kair/git/Backward/examples/UBQ_martini/2xUBQ_cg.pdb" # coarse grained structure for backmapping
aa_topology  = "/home/kair/git/Backward/examples/UBQ_martini/2xUBQ_aa.top" # atomistic topology

# output
aa_structure_raw = "2xUBQ_aa_raw.gro" # initial structure after running backward.py
aa_structure     = "2xUBQ_aa.gro" # final structure after energy min.

## Backmapping

In [24]:
args = '-f {} -p {} -o {} -to gromos -kick 0.05'.format(cg_structure, aa_topology, aa_structure_raw)
args_list = args.split()

In [25]:
backward.workflow(args_list)

Error reading charmm27 to martini mapping for POPG (file: /home/kevin/projects/tobias_schneider/Mapping/pops.amber.map).
Error reading charmm36 to martini mapping for POPG (file: /home/kevin/projects/tobias_schneider/Mapping/pops.amber.map).
Error reading slipids to martini mapping for POPG (file: /home/kevin/projects/tobias_schneider/Mapping/pops.amber.map).
Error reading amber to martini mapping for POPG (file: /home/kevin/projects/tobias_schneider/Mapping/pops.amber.map).
Error reading charmm27 to martini mapping for DOPE (file: /home/kevin/projects/tobias_schneider/Mapping/dopg.charmm36.map).
Error reading charmm36 to martini mapping for DOPE (file: /home/kevin/projects/tobias_schneider/Mapping/dopg.charmm36.map).
Error reading slipids to martini mapping for DOPE (file: /home/kevin/projects/tobias_schneider/Mapping/dopg.charmm36.map).
Error reading amber to martini mapping for DOPE (file: /home/kevin/projects/tobias_schneider/Mapping/dopg.charmm36.map).
Error reading charmm27 to ma

In [26]:
args_list

[]

## OpenMM

In [27]:
gro_raw = app.gromacsgrofile.GromacsGroFile(aa_structure_raw)
top_aa = CustomGromacsTopFile(aa_topology,
            periodicBoxVectors=gro_raw.getPeriodicBoxVectors(),
            includeDir='/home/soft/gromacs/gromacs-2020/inst/cuda_shared_18.04/share/gromacs/top/')
#             includeDir='/home/kair/hiwi/KR0018/example_sim_kevin/')
system = top_aa.createSystem()
integrator = mm.LangevinIntegrator(300*u.kelvin, 1/u.picosecond,0.002*u.picoseconds)
plat = mm.Platform.getPlatformByName('CPU')
simulation = app.Simulation(top_aa.topology, system, integrator, platform=plat)

('/home/kair/git/Backward/examples/UBQ_martini', '/home/soft/gromacs/gromacs-2020/inst/cuda_shared_18.04/share/gromacs/top/', '/home/kair/git/Backward/examples/UBQ_martini')
('/home/kair/git/Backward/examples/UBQ_martini', '/home/soft/gromacs/gromacs-2020/inst/cuda_shared_18.04/share/gromacs/top/', '/home/soft/gromacs/gromacs-2020/inst/cuda_shared_18.04/share/gromacs/top/gromos54a7.ff')
('/home/kair/git/Backward/examples/UBQ_martini', '/home/soft/gromacs/gromacs-2020/inst/cuda_shared_18.04/share/gromacs/top/', '/home/soft/gromacs/gromacs-2020/inst/cuda_shared_18.04/share/gromacs/top/gromos54a7.ff')
('/home/kair/git/Backward/examples/UBQ_martini', '/home/soft/gromacs/gromacs-2020/inst/cuda_shared_18.04/share/gromacs/top/', '/home/soft/gromacs/gromacs-2020/inst/cuda_shared_18.04/share/gromacs/top/gromos54a7.ff')
('/home/kair/git/Backward/examples/UBQ_martini', '/home/soft/gromacs/gromacs-2020/inst/cuda_shared_18.04/share/gromacs/top/', '/home/kair/git/Backward/examples/UBQ_martini')
('/h

In [28]:
simulation.context.setPositions(gro_raw.positions)

In [29]:
state = simulation.context.getState(getPositions=True, getEnergy=True, getForces=True)
state.getPotentialEnergy()#.in_units_of(kilocalorie_per_mole)

Quantity(value=155105527475.53342, unit=kilojoule/mole)

In [30]:
simulation.minimizeEnergy(tolerance=1000, maxIterations=1)

OpenMMException: Particle coordinate is nan

In [19]:
state = simulation.context.getState(getPositions=True, getEnergy=True, getForces=True)
state.getPotentialEnergy()

OpenMMException: Particle coordinate is nan

In [11]:
state = simulation.context.getState(getPositions=True, getEnergy=True, getForces=True)
forces = np.array(state.getForces()._value)
len_forces = np.linalg.norm(forces, axis=1)
np.where(len_forces > 100000)[0]

array([  35,   36,   37,   39,   40,   92,   94,  165,  166,  179,  183,
        328,  329,  349,  353,  356,  360,  366,  367,  401,  402,  404,
        436,  438,  439,  522,  523,  525,  572,  573,  576,  672,  674,
        678,  713,  714,  716,  739,  740,  742,  797,  798,  799,  801,
        802,  814, 1018, 1078, 1107, 1111, 1115, 1118, 1122, 1128, 1129,
       1163, 1164, 1166, 1196, 1197, 1198, 1200, 1201, 1284, 1285, 1287,
       1298, 1300, 1302, 1434, 1436, 1440, 1475, 1476, 1478, 1501, 1502,
       1504])

## Minimizer with fixed distance

In [ ]:
def minimize_fix_dist(simulation, dist_to_move = 0.01 * u.angstrom, steps=5, max_force=500):
    
    energies = []
    
    for _ in range(steps):
        state = simulation.context.getState(getPositions=True, getEnergy=True, getForces=True)
        energies.append(state.getPotentialEnergy())
        forces = state.getForces(asNumpy=True)
        positions = state.getPositions(asNumpy=True)
        
        len_forces = np.linalg.norm(forces, axis=1)
        ind_to_move = np.where(len_forces > max_force)[0]
        dir_to_move = (forces[ind_to_move] / len_forces[ind_to_move, np.newaxis])._value

        positions[ind_to_move] += dir_to_move * dist_to_move
        
        simulation.context.setPositions(positions)
        
    state = simulation.context.getState(getPositions=True, getEnergy=True, getForces=True)
    energies.append(state.getPotentialEnergy())
    
    return energies
    
# minimize_fix_dist(simulation_em)

---

In [28]:
gro_raw = app.gromacsgrofile.GromacsGroFile(aa_structure_raw)
top_aa = CustomGromacsTopFile(aa_topology,
            periodicBoxVectors=gro_raw.getPeriodicBoxVectors(),
            includeDir='/home/soft/gromacs/gromacs-2020/inst/cuda_shared_18.04/share/gromacs/top/')
#             includeDir='/home/kair/hiwi/KR0018/example_sim_kevin/')
system = top_aa.createSystem(nonbondedMethod=app.forcefield.PME, 
                             nonbondedCutoff=2 * u.nanometer,
                             removeCMMotion=False)
integrator_GDM = GradientDescentMinimizationIntegrator(0.5*u.angstrom)
plat = mm.Platform.getPlatformByName('CUDA')
simulation_em = app.Simulation(top_aa.topology, system, integrator_GDM, platform=plat)
simulation_em.context.setPositions(gro_raw.positions)

('', '/home/soft/gromacs/gromacs-2020/inst/cuda_shared_18.04/share/gromacs/top/', '')
('', '/home/soft/gromacs/gromacs-2020/inst/cuda_shared_18.04/share/gromacs/top/', '/home/soft/gromacs/gromacs-2020/inst/cuda_shared_18.04/share/gromacs/top/gromos54a7.ff')
('', '/home/soft/gromacs/gromacs-2020/inst/cuda_shared_18.04/share/gromacs/top/', '/home/soft/gromacs/gromacs-2020/inst/cuda_shared_18.04/share/gromacs/top/gromos54a7.ff')
('', '/home/soft/gromacs/gromacs-2020/inst/cuda_shared_18.04/share/gromacs/top/', '/home/soft/gromacs/gromacs-2020/inst/cuda_shared_18.04/share/gromacs/top/gromos54a7.ff')
('', '/home/soft/gromacs/gromacs-2020/inst/cuda_shared_18.04/share/gromacs/top/', '')
('', '/home/soft/gromacs/gromacs-2020/inst/cuda_shared_18.04/share/gromacs/top/', '')
G96 fourth power
cosine angle


In [29]:
state = simulation_em.context.getState(getPositions=True, getEnergy=True, getForces=True)
print('Initial:\t',state.getPotentialEnergy())

simulation_em.step(10000)
state = simulation_em.context.getState(getPositions=True, getEnergy=True, getForces=True)
print('GDM:\t\t',state.getPotentialEnergy())

simulation_em.minimizeEnergy()
state = simulation_em.context.getState(getPositions=True, getEnergy=True, getForces=True)
print('Minimized:\t',state.getPotentialEnergy())

Initial:	 347315257842.3651 kJ/mol
GDM:		 347315257842.3651 kJ/mol


Exception: Particle coordinate is nan

In [109]:
integratior_equil =  mm.LangevinIntegrator(10*u.kelvin, 1/u.picosecond,0.00001*u.picoseconds)
simulation_equil = app.Simulation(simulation_em.topology, simulation_em.system, integratior_equil, platform=plat)
state = simulation_em.context.getState(getPositions=True, getEnergy=True, getForces=True)
simulation_equil.context.setPositions(state.getPositions())

In [161]:
simulation_equil.minimizeEnergy(tolerance=10, maxIterations=0)

state = simulation_equil.context.getState(getPositions=True, getEnergy=True, getForces=True)
state.getPotentialEnergy()#.in_units_of(kilocalorie_per_mole)

Quantity(value=-18921050.060507752, unit=kilojoule/mole)

In [162]:
simulation_equil.step(10000)

state = simulation_equil.context.getState(getPositions=True, getEnergy=True, getForces=True)
state.getPotentialEnergy()#.in_units_of(kilocalorie_per_mole)

Quantity(value=1164667891.9394922, unit=kilojoule/mole)

In [ ]:
with open('test.pdb', 'w') as fp:
    app.PDBFile.writeFile(simulation_equil.topology, state.getPositions(), fp)

In [ ]:
_ = minimize_fix_dist(simulation_equil, 0.2*u.angstrom, steps=1, max_force=10000)
_

In [ ]:
state = simulation_equil.context.getState(getPositions=True, getEnergy=True, getForces=True)
forces = state.getForces(asNumpy=True)
len_forces = np.linalg.norm(forces, axis=1)
np.where(len_forces > 10000)[0]

In [ ]:
state = simulation_equil.context.getState(getPositions=True, getEnergy=True, getForces=True)
state.getPotentialEnergy()#.in_units_of(kilocalorie_per_mole)

## Write bonds into .csv and load them into VMD

```python
with open("bonds.csv", 'w') as fp:
    for bond in top_aa.topology.bonds():
        fp.write('{} {}\n'.format(bond.atom1.index, bond.atom2.index))
```
---
```bash
cat << 'EOF' > load_bonds.tcl
topo clearbonds

set fp [open bonds.csv]
while {[gets $fp line] >= 0} {
    # do what you like here
    topo addbond {*}${line}
}

close $fp
mol reanalyze top
EOF
```

## 'reporter' for energy minimization

In [ ]:
def show_em(simulation, n_iterations, n_report=10):
    list_pe = []
    n_reports = 10
    for i in range(n_iterations//n_report):
        simulation.minimizeEnergy(maxIterations=n_report)
        state = simulation.context.getState(getEnergy=True)
        pe = state.getPotentialEnergy()
        list_pe.append(pe)
    if n_iterations % n_report:
        simulation.minimizeEnergy(maxIterations=n_iterations % n_report)
        state = simulation.context.getState(getEnergy=True)
        pe = state.getPotentialEnergy()
        list_pe.append(pe)
    return list_pe

In [ ]:
list_pe

In [ ]:
state = simulation.context.getState(getEnergy=True)
state.getPotentialEnergy()

In [ ]:
pos = Quantity(np.array(gro_raw.positions.value_in_unit(nanometer)), nanometer)
simulation.context.setPositions(pos)

In [ ]:
state = simulation.context.getState(getPositions=True)
state.getPositions()

---
### Testsystems for forces

In [64]:
test_gro = '/home/kair/Downloads/test_system/init.gro'
test_top = '/home/kair/Downloads/test_system/em_vac.top'
test_trr = '/home/kair/Downloads/test_system/traj.trr'

In [65]:
gro_raw = app.gromacsgrofile.GromacsGroFile(test_gro)
top_aa = CustomGromacsTopFile(test_top,
            periodicBoxVectors=gro_raw.getPeriodicBoxVectors(),
            includeDir='/home/soft/gromacs/gromacs-2020/inst/cuda_shared_18.04/share/gromacs/top/')
system = top_aa.createSystem()
integrator = mm.LangevinIntegrator(300*u.kelvin, 1/u.picosecond,0.002*u.picoseconds)
plat = mm.Platform.getPlatformByName('CUDA')
simulation = app.Simulation(top_aa.topology, system, integrator, platform=plat)

('/home/kair/Downloads/test_system', '/home/soft/gromacs/gromacs-2020/inst/cuda_shared_18.04/share/gromacs/top/', '/home/kair/Downloads/test_system')
('/home/kair/Downloads/test_system', '/home/soft/gromacs/gromacs-2020/inst/cuda_shared_18.04/share/gromacs/top/', '/home/soft/gromacs/gromacs-2020/inst/cuda_shared_18.04/share/gromacs/top/gromos54a7.ff')
('/home/kair/Downloads/test_system', '/home/soft/gromacs/gromacs-2020/inst/cuda_shared_18.04/share/gromacs/top/', '/home/soft/gromacs/gromacs-2020/inst/cuda_shared_18.04/share/gromacs/top/gromos54a7.ff')
('/home/kair/Downloads/test_system', '/home/soft/gromacs/gromacs-2020/inst/cuda_shared_18.04/share/gromacs/top/', '/home/soft/gromacs/gromacs-2020/inst/cuda_shared_18.04/share/gromacs/top/gromos54a7.ff')
('/home/kair/Downloads/test_system', '/home/soft/gromacs/gromacs-2020/inst/cuda_shared_18.04/share/gromacs/top/', '/home/kair/Downloads/test_system')
('/home/kair/Downloads/test_system', '/home/soft/gromacs/gromacs-2020/inst/cuda_shared_1

In [66]:
simulation.context.setPositions(gro_raw.positions)

In [67]:
state = simulation.context.getState(getForces=True, getEnergy=True, getPositions=True)
state.getPotentialEnergy()
len(state.getForces())

308